In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf


In [6]:
train = pd.read_csv('./data/written_name_train_v2.csv')
valid = pd.read_csv('./data/written_name_validation_v2.csv')

In [7]:
train.head

<bound method NDFrame.head of                 FILENAME       IDENTITY
0        TRAIN_00001.jpg      BALTHAZAR
1        TRAIN_00002.jpg          SIMON
2        TRAIN_00003.jpg          BENES
3        TRAIN_00004.jpg        LA LOVE
4        TRAIN_00005.jpg         DAPHNE
...                  ...            ...
330956  TRAIN_330957.jpg          LENNY
330957  TRAIN_330958.jpg        TIFFANY
330958  TRAIN_330959.jpg  COUTINHO DESA
330959  TRAIN_330960.jpg         MOURAD
330960  TRAIN_330961.jpg        HELOISE

[330961 rows x 2 columns]>

In [10]:
#Cleaning the data 
print("Number of train NaN:",train['IDENTITY'].isnull().sum())
print("Number of train NaN:",valid['IDENTITY'].isnull().sum())

Number of train NaN: 565
Number of train NaN: 78


In [11]:
#Dropping NaN Values 
train.dropna(axis=0,inplace=True)
valid.dropna(axis=0,inplace=True)